In [22]:
# !pip install ctransformers

In [23]:
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.llms import CTransformers
from src.helper import *

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

# instruction = "Convert the following text from English to Hindi: \n\n {text}"
instruction = "Give a proper summary of the book?"

CUSTOM_SYSTEM_PROMPT="You are an advanced assistant that provides summary of the book: \n\n {text}"

SYSTEM_PROMPT = B_SYS + CUSTOM_SYSTEM_PROMPT + E_SYS
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

prompt = PromptTemplate(template = template, input_variables = ['text'])

llm = CTransformers(model = '/Users/kushalbanda/Generative AI/Courses/ineuron/LLMs on CPU/model/llama-2-7b-chat.ggmlv3.q4_0.bin',
                model_type = 'llama',
                config = {'max_new_tokens': 128,
                        'temperature': 0.8}
                        )

In [24]:
LLM_Chain = LLMChain(llm = llm, prompt = prompt)

In [25]:
print(LLM_Chain.run("Harry Potter"))

  Certainly! "Harry Potter" is a popular children's book series written by J.K. Rowling. The story follows the adventures of a young wizard named Harry Potter and his friends Ron Weasley and Hermione Granger as they attend Hogwarts School of Witchcraft and Wizardry.

The books are set in a world where magic is real, and follow Harry's journey from being an awkward and unpopular boy to becoming a confident and powerful wizard. Along the way, he must confront the dark wizard Lord Voldemort, who


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from src.helper import *

# Load the PDF File
loader = DirectoryLoader('data/',
                        glob = "*.pdf",
                        loader_cls = PyPDFLoader)

documents = loader.load()

# Split Text into Chunks 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50)

text_chunks = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2',
                                model_kwargs = {'device': 'cpu'})

# Convert the Text Chunks into Embeddings and Create a FAISS vector Store
vector_store = FAISS.from_documents(text_chunks, embeddings)

llm = CTransformers(model = '/Users/kushalbanda/Generative AI/Courses/ineuron/LLMs on CPU/model/llama-2-7b-chat.ggmlv3.q4_0.bin',
                    model_type = 'llama',
                    config = {'max_new_tokens': 128,
                            'temperature': 0.01})

qa_prompt = PromptTemplate(template=template, input_variables=['context', 'question'])

chain = RetrievalQA.from_chain_type(llm = llm,
                                    chain_type = 'stuff',
                                    retriever = vector_store.as_retriever(search_kwargs = {'k': 2}),
                                    return_source_documents = False,
                                    chain_type_kwargs = {'prompt': qa_prompt})

user_input = "Tell me about Rainfall Measurement of the paper"

result = chain({'query': user_input})
print(f"Answer: {result['result']}")


/Users/kushalbanda/miniconda3/envs/Image_Captioning/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
